In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

In [ ]:
detectors = pd.read_csv("UTD19/detectors_public.csv")
links = pd.read_csv("UTD19/links.csv")
measurements = pd.read_csv("UTD19/utd19_u.csv")

In [ ]:
print(measurements.head(5))